# Data Processing:

In [2]:
# Mount into drive

from google.colab import drive

drive.mount("/content/drive")

%cd '/content/drive/MyDrive/Colab Notebooks/Money Printer/'

# !pip install -r requirements.txt
# !pip install pandas
# !pip install -U scikit-learn
# !pip install torchinfo


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Money Printer


In [77]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pathlib import Path


# Data Loader Script

In [78]:
def get_data_loaders(device, label="label_1", batch_size=64, use_custom_cols=True, window_size=10, flatten=True):
    train_data = pd.read_csv("./data/Train_NoAuction_Zscore.csv")
    test_data = pd.read_csv("./data/Test_NoAuction_Zscore.csv")

    X_train = train_data.drop(
        columns=["label_1", "label_2", "label_3", "label_5", "label_10"])
    X_test = test_data.drop(
        columns=["label_1", "label_2", "label_3", "label_5", "label_10"])

    y_train = train_data[label] - 1
    y_test = test_data[label] - 1
    if not use_custom_cols:
        X_train = X_train.iloc[:, :40]
        X_test = X_test.iloc[:, :40]

    X_train = X_train.to_numpy()
    X_test = X_test.to_numpy()
    y_train = y_train.to_numpy()
    y_test = y_test.to_numpy()

    # sliding window
    if window_size != 1:
        D = X_train.shape[1]
        X_train = np.lib.stride_tricks.sliding_window_view(
            X_train, (window_size, D))
        if flatten:
          X_train = X_train.reshape((-1, window_size*D))
        y_train = y_train[window_size-1:]

        X_test = np.lib.stride_tricks.sliding_window_view(
            X_test, (window_size, D)).squeeze()
        if flatten:
            X_test = X_test.reshape((-1, window_size*D))
        y_test = y_test[window_size-1:]

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, shuffle=True, test_size=0.2)

    X_train_tensor = torch.tensor(
        X_train, dtype=torch.float32).to(device)
    X_val_tensor = torch.tensor(
        X_val, dtype=torch.float32).to(device)
    # Unsqueeze here for DeepLOB and TransLOB models:
    X_test_tensor = torch.tensor(
        X_test, dtype=torch.float32).unsqueeze(1).to(device)


    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

# Model Script:

In [79]:
class DeepLOB(nn.Module):
    def __init__(self, n_classes=3):
        super(DeepLOB, self).__init__()
        self.inception_num = 0
        self.n_classes = n_classes

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,10)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )

        # 2. Inception Layer:
        ##1st inc 1x1 3x1
        self.inc1 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inception_num+=1

        ##2nd inc 1x1 5x1
        self.inc2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inception_num+=1

        ##3nd inc max_pool 1x1
        self.inc3 = nn.Sequential(
            nn.MaxPool2d((3, 1), stride=(1, 1), padding=(1, 0)),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inception_num+=1

        # 2. LSTM layers

        self.lstm_in_size = self.inception_num*64
        self.lstm = nn.LSTM(self.lstm_in_size, hidden_size=64, num_layers=1,batch_first=True)
        self.fc1 = nn.Linear(64, self.n_classes)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), 64).to(device)
        c0 = torch.zeros(1, x.size(0), 64).to(device)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        inception1 = self.inc1(x)
        inception2 = self.inc2(x)
        inception3 = self.inc3(x)

        x = torch.cat((inception1, inception2, inception3), dim=1)

        x = x.permute(0, 2, 1, 3)
        x = torch.reshape(x, (-1, x.shape[1], x.shape[2]))

        x, _ = self.lstm(x, (h0, c0))
        x = x[:, -1, :]
        x = self.fc1(x)
        output = torch.softmax(x, dim=1)

        return output

# Training and Testing Script

In [80]:
def test_model(model, test_loader, experiment_name, save=True, display=False):
    test_predictions = []
    test_labels = []

    model.eval()
    with torch.no_grad():
        for batch, (X, y) in enumerate(test_loader):
          pred = model(X)
          _, predicted = torch.max(pred, 1)
          test_predictions.extend(predicted.cpu().numpy())
          test_labels.extend(y.cpu().numpy())
    report = classification_report(test_labels, test_predictions)
    if display:
        print(report)
    if save:
        with open(f"results/{experiment_name}/report.txt", "w") as f:
            f.write(report)


def visualize_curves(train_losses, val_losses, train_accuracies, val_accuracies, experiment_name, save=True, display=False):
    plt.plot(np.arange(epochs), train_losses, label="train")
    plt.plot(np.arange(epochs), val_losses, label="val")
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.title(f"{experiment_name} loss curves")
    plt.legend()
    if display:
        plt.show()
    if save:
        plt.savefig(f"results/{experiment_name}/loss_curves.png")
    plt.close()

    plt.plot(np.arange(epochs), train_accuracies, label="train")
    plt.plot(np.arange(epochs), val_accuracies, label="val")
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.title(f"{experiment_name} accuracy curves")
    plt.legend()
    if display:
        plt.show()
    if save:
        plt.savefig(f"results/{experiment_name}/accuracy_curves.png")
    plt.close()


def train_and_evaluate_model(train_loader, val_loader, test_loader, experiment_name, optimizer, criterion, epochs):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for i in range(epochs):
        model.train()
        running_train_loss = 0
        train_correct_predictions = 0
        train_total_samples = 0
        for batch, (X, y) in enumerate(train_loader):
            pred = model(X)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            running_train_loss += loss.item()

            _, predicted = torch.max(pred, 1)
            train_correct_predictions += (predicted == y).sum().item()
            train_total_samples += y.size(0)

        print(f"Epoch {i} train loss {running_train_loss/len(train_loader)}")
        train_losses.append(running_train_loss/len(train_loader))

        train_accuracy = train_correct_predictions / train_total_samples
        train_accuracies.append(train_accuracy)
        print(f"Epoch {i} train accuracy {train_accuracy}")

        val_correct_predictions = 0
        val_total_samples = 0
        model.eval()
        with torch.no_grad():
            running_val_loss = 0
            for batch, (X, y) in enumerate(val_loader):
                pred = model(X)
                loss = criterion(pred, y)
                running_val_loss += loss.item()

                _, predicted = torch.max(pred, 1)
                val_correct_predictions += (predicted == y).sum().item()
                val_total_samples += y.size(0)

            print(f"Epoch {i} val loss {running_val_loss/len(val_loader)}")
            val_losses.append(running_val_loss/len(val_loader))

            val_accuracy = val_correct_predictions / val_total_samples
            val_accuracies.append(val_accuracy)
            print(f"Epoch {i} val accuracy {val_accuracy}")

    test_model(model, test_loader, experiment_name)
    visualize_curves(train_losses, val_losses,
                     train_accuracies, val_accuracies, experiment_name)

In [ ]:
# get device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# hyperparameters
learning_rate = 1e-4
batch_size = 64
epochs = 2
window_size = 100

# model definition
# model = MLP(input_size=NUM_FEATURES*window_size, hidden_size=64)
model = DeepLOB(n_classes = 3)
model = model.to(device)

# experiment name
experiment_name = "deeplob-s10"

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for label in ["label_1", "label_2", "label_3", "label_5", "label_10"]:
    new_experiment_name = experiment_name + f"/{label}"
    Path(
        f"results/{new_experiment_name}").mkdir(parents=True, exist_ok=True)

    # get data loaders
    train_loader, val_loader, test_loader = get_data_loaders(device=device, window_size=100, batch_size=64,
                                                              flatten=False,use_custom_cols=False)
    #unsqueeze test loader for deepLob:
    # test_loader = test_loader.unsqueeze(1)

    # training and evaluation
    train_and_evaluate_model(train_loader, val_loader,
                              test_loader, new_experiment_name, optimizer, criterion, epochs)

Using cuda device
Epoch 0 train loss 0.9760156907360774
Epoch 0 train accuracy 0.5771335464830301
Epoch 0 val loss 1.0540281208232045
Epoch 0 val accuracy 0.4223533751383253
Epoch 1 train loss 0.9456845573451523
Epoch 1 train accuracy 0.5966244466305952
Epoch 1 val loss 1.1372333243489265
Epoch 1 val accuracy 0.2032460346735522
Epoch 0 train loss 0.9457831437789153
Epoch 0 train accuracy 0.5964707329070339
Epoch 0 val loss 1.056595423258841
Epoch 0 val accuracy 0.49944669863519
Epoch 1 train loss 0.9443229412283083
Epoch 1 train accuracy 0.5963785046728972
Epoch 1 val loss 0.9713797788135707
Epoch 1 val accuracy 0.6000245911717693
Epoch 0 train loss 0.946008792092149
Epoch 0 train accuracy 0.5962862764387604
Epoch 0 val loss 1.021754251793027
Epoch 0 val accuracy 0.4436247387187999
Epoch 1 train loss 0.9457716514413156
Epoch 1 train accuracy 0.5953025086079685
Epoch 1 val loss 0.9481913107447326
Epoch 1 val accuracy 0.6029755317840895
Epoch 0 train loss 0.9432468259732709
Epoch 0 train

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 0 train loss 0.9410916301028434
Epoch 0 train accuracy 0.5997909493359567
Epoch 0 val loss 1.0666650994680822
Epoch 0 val accuracy 0.5639985245296938
Epoch 1 train loss 0.9404144284767346
Epoch 1 train accuracy 0.6005287752090507
Epoch 1 val loss 0.9758689273148775
Epoch 1 val accuracy 0.5719906553547277
